In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
from matplotlib.font_manager import FontProperties

fp = FontProperties(fname=r'/system/library/fonts/ヒラギノ角ゴシック W0.ttc', size = 14)

/Users/yamamotomasaomi/.pyenv/versions/anaconda3-5.1.0/envs/makuwo/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mnist = input_data.read_data_sets("/tmp/data", one_hot=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# images, labels = mnist.train.next_batch(10)

In [4]:
# print (images[0])

In [5]:
# print (labels)

In [6]:
%config IPCompleter.greedy=True
%matplotlib inline
%matplotlib notebook

fig = plt.figure(figsize=(8,4))
for c, (image,label) in enumerate(zip(images,labels)):
    subplot = fig.add_subplot(2,5,c+1)
    subplot.set_xticks([])
    subplot.set_yticks([])
    subplot.set_title('%d' % label)
    subplot.imshow(image.reshape((28,28)), vmin = 0, vmax=1, cmap=plt.cm.gray_r, interpolation="nearest")

In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import multivariate_normal,permutation
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160512)

In [8]:
mnist = input_data.read_data_sets("/tmp/data", one_hot=False)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [9]:
x = tf.placeholder(tf.float32,[None, 784])
w = tf.Variable(tf.zeros([784, 1]))
w0 = tf.Variable(tf.zeros([1]))
f = tf.matmul(x, w) + w0
# p = tf.nn.softmax(f)
p = tf.sigmoid(f)

In [10]:
t = tf.placeholder(tf.float32, [None, 1])
# loss = -tf.reduce_sum(t * tf.log(p))
loss = -tf.reduce_sum(t*tf.log(p) + (1-t)*tf.log(1-p))
train_step = tf.train.AdamOptimizer().minimize(loss)

In [11]:
# correct_prediction = tf.equal(tf.sign(p-0.5),tf.sign(t-0.5))
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
i=0
batch_xs, batch_ts = mnist.train.next_batch(100)
batch_ts = (batch_ts ==2)
batch_ts = batch_ts.reshape(100,1).astype(int)
p_val = sess.run(p, feed_dict={x:batch_xs, t:batch_ts})

for _ in range(1000):
    i += 1
    for (image, label, pred) in zip(batch_xs, batch_ts, p_val):
        prediction, actual = np.argmax(pred), label
        sess.run(train_step, feed_dict={x:batch_xs,t:batch_ts})
#     if  batch_ts[i]==2:
#         print (prediction)
#         print (batch_ts[i])
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss,accuracy],feed_dict={x:batch_xs,t:batch_ts})
        print ('Step:%d, Loss:%f, Accuracy:%f' % (i, loss_val, acc_val) )
        print (prediction)
        print (batch_ts[i])

In [ ]:
train_accuracy = []
test_accuracy = []
train_y = []
test_y = []
for _ in range (2500): 
    sess.run(train_step,feed_dict={x:batch_xs,t:batch_ts})
    loss_val =sess.run(loss, feed_dict={x:batch_xs,t:batch_ts})
    train_y.append(loss_val)
#     loss_val =sess.run(loss, feed_dict={x:test_x,t:test_t})
#     test_y.append(loss_val)
    
    acc_val =sess.run(accuracy, feed_dict={x:batch_xs,t:batch_ts})
    train_accuracy.append(acc_val)
#     acc_val =sess.run(accuracy, feed_dict={x:test_x,t:test_t})
#     test_accuracy.append(acc_val)

# 復習
先週のは、Mnistを使った分類法について一応分類して見ましょうと言う話ではあるが
とにかく、Mnistを触ってみようと言う意味合いでやって見た次第というわけ
これから先、別のも動かしてみようということになった
mnistをもちいて、これまでのロジスティック回帰をしてもらおうということになったのだ

例えば、０かそうでないかという分岐をしてみよう
０か否か（正答率に関する話ということなので、選択肢が２択であれば、問題はない。今までのデータをもちいて、できるはずなので、やってみよう。）

Mnistの中では、全部で7万もの筆跡のデータが入っているんです。
訓練用（学習用）:train 55000個
テスト用:test 10000個
バリエーション:variation 5000個
無論すべてを使う事はできるが、わけて使うほうが普通のやり方（過学習の予防）

# 取り出し方
mds-02にて、mnistが取り出されている
mnist.train(クラスと同じ扱い)

以下のようなオプションが存在する
image => 画像
label =>　画像の何が書かれているかのデータ

tarain = mnist.train.images => 配列の中みすべてを表す
55000通りのベクトルを示している。

one_hot => ラベルをどういう具合に表現するのか？=>ひとつひとつが１０個のベクトルとして表される
 True: [0,0,1,0,0,0,0,0,0,0]
false:3

テキストはTrueとなっている。
今はFalseにすれば、問題の条件式を作れるはず。

<b>この中で数字が0かどうかでラベルをつけ直す</b>
train0_labels=>(train.labels == 0)
true,falseを返すようにする。

何個はいってる？＝＞trueを何個あるかを出せば良い

np.sum(train0_labels) => 1の個数が出る

mle-05(p.73)
x=tf.placeholder [None.2] =>感染非感染で２にしてた。しかし、ベクトルなので784次元となるはずだ。つまり、それらの数値も次元を理解してないと意味がない。
シグモイド関数を使用して、
t = tf.placeholder([None,1])でよい。
optimizerもそのまま。学習すれば良い。

学習回数は、2万回でやってるが、今回は、1000回くらいで平気。

# 評価
今、trainで話をしたが、testも用意する事。学習した評価をtestで行う事。
こちらも、以前と同じプログラムを流用する形。
テストする数は10000。
2という文字について、テストの1万個の中に、本当に2というものと、本当に2でないというものがある。
予想が2である事と2でない判定がある。
それぞれを表で表すため、４通りになる。

```
本当に０でない　＝　０でない　＝＞真陽性（TN）
０である　＝　０である　＝＞真陽性（TN）
本当に０でない　＝　０である　＝＞　偽陽性（PP）
０である　＝＞　０でない　＝＞　偽陰性（PN）
```
## 混同行列
recall = TP/FN＋TP 再現率
prediction_2 = TP/Tp+FP

prediction = (p>=0.5)
tensor flowで定義したものなので、使える。学習した後、求めてやると、判定結果が出るように設定する。
sess.run(prediction, feed_dict ={x:train.image t:test.label})

Andでつなげてあげれば、IF文は完成する。

reshapeするのは２次元のベクトルに変換するため。縦ベクトル（何行✖️1にするため。Tの定義と合わないので）
つまり、test.labelは、reshapeされる必要がある。[-1,1]にしておく事で勝手に計算してくれる。

判定結果のTrue,Falseが返されるので、

PCパームレストと、猫の旅雑誌

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.title('ロジスティック回帰による二項分類器',fontproperties=fp)
subplot = fig.add_subplot(1,2,1)
subplot.plot(range (len(train_accuracy)),train_accuracy,linewidth=2,label='Training set')
# subplot.plot(range (len(test_accuracy)),test_accuracy,linewidth=2,label='Test set')
plt.legend([u'トレーニングケース', 'テストケース'], prop=fp, loc='lower right')
fig.xlabel=(u'epoch')
fig.ylabel=(u'accutacy')


subplot2 = fig.add_subplot(1,2,2)
subplot2.plot(range(len(train_y)),train_y,linewidth=2,label='トレーニングセット')
# subplot2.plot(range(len(test_y)),test_y,linewidth=2,label='テストセット')
plt.legend([u'トレーニングケース', 'テストケース'], prop=fp, loc='upper right')
plt.xlabel=(u'epoch')
plt.ylabel=(u'loss/sample')
# subplot.legend(loc='upper left')

# plt.show()

In [ ]:
import csv
with open('file.csv', 'wt') as f:
    writer = csv.writer(f)
    writer.writerows(batch_ts)

In [ ]:
w0_val, w_val= sess.run([w0,w])
w0_val, w1_val, w2_val = w0_val[0], w_val[0][0], w_val[1][0]
print (w0_val, w1_val, w2_val)

In [ ]:
batch_ts[2]